In [ ]:
import pandas as pd
import numpy as np
import datetime
from os import listdir
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [ ]:
df = pd.read_csv("properatti.csv", encoding='utf-8')

In [ ]:
df.columns

In [ ]:
df.rename(columns={'Unnamed: 0':'ID'}, inplace=True)

In [ ]:
df.drop(df.columns[[1,4,7,22,25]], axis=1, inplace=True)

In [ ]:
import re


In [ ]:
#df_apartments = df[(df["property_type"]=="apartment")]

In [ ]:
#df_cf = df[(df["state_name"]=="Capital Federal")]
df_apart = df[(df["property_type"]=="apartment") & (df["state_name"]=="Capital Federal")]
df_apart

In [ ]:
df_apart.to_csv('apartments.csv', encoding='utf-8')

In [ ]:
'''hasta aqui se realizo la separacion de los departamentos de capital federal y se guardo en un archivo csv
recomiendo cargar desde aqui en adelante'''

In [ ]:
import pandas as pd
import numpy as np
import datetime
from os import listdir
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [ ]:

df_apartments = pd.read_csv("apartments.csv", encoding='utf-8')
df_apartments["title"] = df_apartments["title"].str.lower()
df_apartments["description"] = df_apartments["description"].str.lower()
pattern = '([0-9])+\s+dorm'
df_apartments

In [ ]:
'''Defino la regex y busco en el título. Luego reemplazo los grupos de captura por valores útiles'''
import re
pattern_amb = '([0-9])+\s+amb|(monoamb)|(dos)\s+amb|([0-9])+\s+dorm|(un)\s+amb|(mono\samb)'
amb_title = df_apartments["title"].str.extract(pattern_amb,flags=re.IGNORECASE,expand=False)
amb_title[1].replace("monoamb", '1', regex=True, inplace=True)
amb_title[2].replace("dos", '2', regex=True, inplace=True)
amb_title[4].replace("un", '1', regex=True, inplace=True)
amb_title[5].replace("mono amb", '1', regex=True, inplace=True)
amb_title

In [ ]:
'''Definicion de funcion de búsqueda del mejor dato encontrado por la regex de Ambientes'''
def selectAmbientes(x):
    if x[0] is not np.nan:
        return x[0]
    if x[1] is not np.nan:
        return x[1]
    if x[2] is not np.nan:
        return x[2]
    if x[3] is not np.nan:
        return x[3]
    if x[4] is not np.nan:
        return x[4]
    if x[5] is not np.nan:
        return x[5]
    else:
        return np.nan

In [ ]:
'''obtengo una serie con el resultado de colapsar por el mejor resultado de la búsqueda Regex del título'''
amb_title_collapse = amb_title.apply(selectAmbientes,axis=1)
#amb_title_collapse.head(20)

In [ ]:
'''Busco la regex en la descripcion'''
amb_descr = df_apartments["description"].str.extract(pattern_amb,flags=re.IGNORECASE,expand=False)
amb_descr[1].replace("monoamb", '1', regex=True, inplace=True)
amb_descr[2].replace("dos", '2', regex=True, inplace=True)
amb_descr[4].replace("un", '1', regex=True, inplace=True)
amb_descr[5].replace("mono amb", '1', regex=True, inplace=True)
amb_descr_collapse = amb_descr.apply(selectAmbientes,axis=1)
#amb_descr_collapse.head(20)

In [ ]:
'''Defino un DF a partir de las series rooms del DF orifinal, resultado regex de descripcion y título'''
df_rooms_full = pd.DataFrame({'orig':df_apartments["rooms"],
                             'description':amb_descr_collapse,
                             'title':amb_title_collapse
                             })
#df_rooms_full

In [ ]:
'''Defino la funcion para buscar el mejor valor entre los encontrados en original, título o descripcion
con orden de importancia título, descripción y de no encontrar, original'''

def guardarDescripcionAmbiente(x):
    if not x['title'] is np.nan:
        return x['title']
    if not x['description'] is np.nan:
        return x['description']
    return x['orig']
    

In [ ]:
'''Reemplazo la columna rooms por los datos minados'''
df_apartments["rooms"] = df_rooms_full.apply(guardarDescripcionAmbiente,axis=1)

In [ ]:
df_apartments.rooms.unique()

In [ ]:
'''Normalizo los valores reemplazando los float para tener todo expresado de la misma forma'''
float_nums = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,14.0,17.0,22.0]
str_nums = ['1','2','3','4','5','6','7','8','9','10','11','12','14','17','22']
df_apartments["rooms"].replace(float_nums,str_nums, inplace=True)

In [ ]:
'''Verifico que se hayan unificado los valores'''
df_apartments.rooms.unique()

In [ ]:
'''me fijo la cantidad de nulos restantes en la columna rearmada por la minería
se han recuperado unos 10000 rows que antes eran nulos'''
df_apartments[(df_apartments["rooms"].isnull())]

In [ ]:
'''creo una transformacion para limpiar datos que no puedan ser valor final de la propiedad,
o sea superiores a 10000 dolares'''
def priceClean(x):
    if x is not np.nan:
        if int(x) > 10000:
            return int(x)
        else:
            return 0
    else:
        return 0

In [ ]:
'''creo un patron de busqueda para obtener valor de propiedad en dolares en el título'''
pattern_price = '[us?\$?d?s?]{3}\s?([0-9]{2,4}\.?\,?[0-9]+)'
price_title = df_apartments["title"].str.extract(pattern_price,flags=re.IGNORECASE,expand=False)
price_title.replace('\.','', regex=True, inplace=True)
price_title.replace('\,','', regex=True, inplace=True)
price_title = price_title.apply(priceClean)
price_title[price_title!=0].head()

In [ ]:
price_desc = df_apartments["description"].str.extract(pattern_price,flags=re.IGNORECASE,expand=False)
price_desc.replace('\.','', regex=True, inplace=True)
price_desc.replace('\,','', regex=True, inplace=True)
price_desc = price_desc.apply(priceClean)
price_desc[price_desc!=0].head()

In [ ]:
df_price_full = pd.DataFrame({'orig':df_apartments['price'],
                             'price_title':price_title,
                             'price_desc':price_desc})
df_price_full[df_price_full['orig'].isnull() & ((df_price_full['price_title']!=0) | (df_price_full['price_desc']!=0))].head()

In [ ]:
def priceAssign(x):
    if np.isnan(x['orig']):
        if x["price_title"]!=0:
            return x["price_title"]
        else:
            if x["price_desc"]!=0:
                return x["price_desc"]
            else:
                return x['orig']
    else:
        return x['orig']

df_apartments["price"] = df_price_full.apply(priceAssign,axis=1)
df_apartments[(df_apartments["currency"]=='ARS')]

In [ ]:
df_apartments[df_apartments["surface_total_in_m2"].isnull()]

In [ ]:
'''hago una transformacion para colocar los metros cubiertos en los casos que la superficie total es Nula
de esta manera pasamos de 4457 valores no reportados a 529'''
def checkSurface(x):
    if np.isnan(x["surface_total_in_m2"]):
        return x["surface_covered_in_m2"]
    else:
        return x["surface_total_in_m2"]

df_apartments["surface_total_in_m2"] = df_apartments.apply(checkSurface,axis=1)
df_apartments

In [ ]:
'''convierto todas las monedas a dolares y me deshago de los valores que no poseen precio o superficie total'''
dolar = 19.5
'''se estima el dolar de ver la conversion de precio por m2 en moneda local vs usd'''
df_apartments.loc[df_apartments['currency'] == 'ARS', 'price'] = df_apartments['price'] / dolar
df_apartments.loc[df_apartments['currency'] == 'ARS', 'price_usd_per_m2'] = (df_apartments['price'] / df_apartments['surface_total_in_m2'])

df_apartments.loc[df_apartments['currency'] == 'ARS', 'currency'] = 'USD'
df_a_clean = df_apartments[(df_apartments["price"].notnull()) &
                           (df_apartments["surface_total_in_m2"].notnull()) &
                           (df_apartments["surface_total_in_m2"]>0) &
                           (df_apartments["currency"].notnull())]
df_a_clean

In [ ]:
'''creo una funcion de calculo para los nan de precio x m2'''
def calculateM2(x):
    if np.isnan(x["price_usd_per_m2"]):
        return (x["price"]/x["surface_total_in_m2"])
    else:
        return x["price_usd_per_m2"]


In [ ]:
'''construyo el valor de precio x m2 a partir del precio y metros totales'''
df_a_clean["price_usd_per_m2"] = df_a_clean.apply(calculateM2,axis=1)
df_a_clean

In [ ]:
'''me fijo si los valores de price son consistentes con metros totales y precio de m2, de no serlo'''
def checkPrices(x):
    est_price = (x["price_usd_per_m2"]*x["surface_total_in_m2"])
    est_error = est_price*0.01
    if ((x["price"] < (est_price + est_error)) & (x["price"] > (est_price - est_error))):
        return True
    else:
        return False

df_a_clean["price_consistent"] = df_a_clean.apply(checkPrices,axis=1)
df_a_clean[(df_a_clean["price_consistent"]==False)]

In [ ]:
df_a_clean.isnull().sum()

In [ ]:
'''guardo el dataframe a disco como limpio con 23764 registros utilizables'''
df_a_clean.to_csv("apartments_clean.csv")


In [1]:
'''Desde aqui se comienza a operar con el csv nuevo'''
import pandas as pd
import numpy as np
import datetime
from os import listdir
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [2]:
df_limpio = pd.read_csv("apartments_clean.csv", encoding='utf-8')
df_limpio

,Unnamed: 0,Unnamed: 0.1,ID,property_type,place_name,country_name,state_name,lat-lon,lat,lon,...,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,price_consistent
0,0,2,2,apartment,Mataderos,Argentina,Capital Federal,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,55.0,1309.090909,1309.090909,NaN,2.0,NaN,"2 ambientes 3er piso lateral living comedor amplio cocina separada con lavadero, baño completo y dormitorio con placard. bajas expensas, excelente ubicacion aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).",2 amb 3er piso con ascensor apto credito,True
1,1,7,7,apartment,Belgrano,Argentina,Capital Federal,"-34.5598729,-58.443362",-34.559873,-58.443362,...,45.0,40.0,3066.666667,3450.000000,NaN,1.0,NaN,"excelente monoambiente a estrenar amplio super luminoso ...piso alto cocina americana. baño completo pileta ,solarium, laundry apto profesionalproximo av del libertador muy buena zona !!!! aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).","jose hernandez 1400 monoambiente estrenar categoria , venta",True
2,2,8,8,apartment,Belgrano,Argentina,Capital Federal,"-34.5598729,-58.443362",-34.559873,-58.443362,...,65.0,60.0,3000.000000,3250.000000,NaN,2.0,NaN,"excelente dos ambientes estrenar amplio super luminoso ...piso alto cocina americana. baño completo pileta ,solarium, laundry apto profesionalproximo av del libertador muy buena zona !!!! aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).","jose hernandez 1400 dos ambientes estrenar , categoria , venta",True
3,3,13,13,apartment,Palermo Soho,Argentina,Capital Federal,NaN,NaN,NaN,...,50.0,30.0,2234.000000,3723.333333,NaN,3.0,NaN,"torre i mondrian. 3 ambientes con terraza y dos baños al frente.vitraux ofrece en el ultimo nivel, piscina con solarium, un espacio con sala de reunión para uso profesional, eventos o proyecciones con un amplio foyer, sauna, gimnasio, un salón de usos múltiples con cocina y una amplia expansión. asimismo cuenta con un hall en doble altura y un amplio estar que balconea al mismo en el primer piso.así destaca la excelente relación de espacios de uso común sobre el metro propio, sobre todo en studios que rondan los 30 metros propios y pueden gozar de todos los diferentes usos especiales.",vitraux palermo,True
4,4,14,14,apartment,Palermo Soho,Argentina,Capital Federal,NaN,NaN,NaN,...,42.0,31.0,3521.428571,4770.967742,NaN,1.0,NaN,"torre ii dalí. ambiente unico divisible.vitraux ofrece en el ultimo nivel, piscina con solarium, un espacio con sala de reunión para uso profesional, eventos o proyecciones con un amplio foyer, sauna, gimnasio, un salón de usos múltiples con cocina y una amplia expansión. asimismo cuenta con un hall en doble altura y un amplio estar que balconea al mismo en el primer piso.así destaca la excelente relación de espacios de uso común sobre el metro propio, sobre todo en studios que rondan los 30 metros propios y pueden gozar de todos los diferentes usos especiales.",vitraux palermo,True
5,5,19,19,apartment,Palermo,Argentina,Capital Federal,"-34.580503566,-58.4058744847",-34.580504,-58.405874,...,104.0,96.0,3365.384615,3645.833333,NaN,3.0,NaN,excelente semipiso al contra frente en bulnes y libertador. espectacular pulmón muy luminoso con árboles. buen edificio con entrada reciclada hace dos años. hall de entrada. living comedor de muy buenas dimensiones con salida al balcón aterrazado. toilette de recpeción. master suite muy grande con vestidor y gran baño (revestimientos de marmol) también al frente. segundo dormitorio o escritorio (chico) a aire luz muy luminoso. muy linda cocina con muebles alemanes. pisos de parquet. persianas eléctricas. aislación acústica en paredes. detalles de muy buena calidad. baulera común. la unidad originalmente era de dos dormitorios pero fue refaccionada al plano actual (master suite más dormitorio chico o escritorio). impecable!,"bulne

In [3]:
#busco parrillas en el titulo y la desc
parrilla_desc = df_limpio["description"].str.contains('parrilla')
parrilla_titl = df_limpio["title"].str.contains('parrilla')

In [4]:
def guardarParrillas(x):
    if x['parrilla_en_el_titulo'] ==True:
        return x['parrilla_en_el_titulo']
    if x['parrilla_en_desc'] == True:
        return x['parrilla_en_desc']
    else:
        return False

In [7]:
parrilla_junto = pd.DataFrame({'parrilla_en_desc':parrilla_desc,
                             'parrilla_en_el_titulo':parrilla_titl
                             })
df_limpio["parrilla"] = parrilla_junto.apply(guardarParrillas,axis=1)
df_limpio

,Unnamed: 0,Unnamed: 0.1,ID,property_type,place_name,country_name,state_name,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,price_consistent,parrilla
0,0,2,2,apartment,Mataderos,Argentina,Capital Federal,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,2.0,NaN,"2 ambientes 3er piso lateral living comedor amplio cocina separada con lavadero, baño completo y dormitorio con placard. bajas expensas, excelente ubicacion aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).",2 amb 3er piso con ascensor apto credito,True,False
1,1,7,7,apartment,Belgrano,Argentina,Capital Federal,"-34.5598729,-58.443362",-34.559873,-58.443362,...,40.0,3066.666667,3450.000000,NaN,1.0,NaN,"excelente monoambiente a estrenar amplio super luminoso ...piso alto cocina americana. baño completo pileta ,solarium, laundry apto profesionalproximo av del libertador muy buena zona !!!! aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).","jose hernandez 1400 monoambiente estrenar categoria , venta",True,False
2,2,8,8,apartment,Belgrano,Argentina,Capital Federal,"-34.5598729,-58.443362",-34.559873,-58.443362,...,60.0,3000.000000,3250.000000,NaN,2.0,NaN,"excelente dos ambientes estrenar amplio super luminoso ...piso alto cocina americana. baño completo pileta ,solarium, laundry apto profesionalproximo av del libertador muy buena zona !!!! aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).","jose hernandez 1400 dos ambientes estrenar , categoria , venta",True,False
3,3,13,13,apartment,Palermo Soho,Argentina,Capital Federal,NaN,NaN,NaN,...,30.0,2234.000000,3723.333333,NaN,3.0,NaN,"torre i mondrian. 3 ambientes con terraza y dos baños al frente.vitraux ofrece en el ultimo nivel, piscina con solarium, un espacio con sala de reunión para uso profesional, eventos o proyecciones con un amplio foyer, sauna, gimnasio, un salón de usos múltiples con cocina y una amplia expansión. asimismo cuenta con un hall en doble altura y un amplio estar que balconea al mismo en el primer piso.así destaca la excelente relación de espacios de uso común sobre el metro propio, sobre todo en studios que rondan los 30 metros propios y pueden gozar de todos los diferentes usos especiales.",vitraux palermo,True,False
4,4,14,14,apartment,Palermo Soho,Argentina,Capital Federal,NaN,NaN,NaN,...,31.0,3521.428571,4770.967742,NaN,1.0,NaN,"torre ii dalí. ambiente unico divisible.vitraux ofrece en el ultimo nivel, piscina con solarium, un espacio con sala de reunión para uso profesional, eventos o proyecciones con un amplio foyer, sauna, gimnasio, un salón de usos múltiples con cocina y una amplia expansión. asimismo cuenta con un hall en doble altura y un amplio estar que balconea al mismo en el primer piso.así destaca la excelente relación de espacios de uso común sobre el metro propio, sobre todo en studios que rondan los 30 metros propios y pueden gozar de todos los diferentes usos especiales.",vitraux palermo,True,False
5,5,19,19,apartment,Palermo,Argentina,Capital Federal,"-34.580503566,-58.4058744847",-34.580504,-58.405874,...,96.0,3365.384615,3645.833333,NaN,3.0,NaN,excelente semipiso al contra frente en bulnes y libertador. espectacular pulmón muy luminoso con árboles. buen edificio con entrada reciclada hace dos años. hall de entrada. living comedor de muy buenas dimensiones con salida al balcón aterrazado. toilette de recpeción. master suite muy grande con vestidor y gran baño (revestimientos de marmol) también al frente. segundo dormitorio o escritorio (chico) a aire luz muy luminoso. muy linda cocina con muebles alemanes. pisos de parquet. persianas eléctricas. aislación acústica en paredes. detalles de muy buena calidad. baulera común. la unidad originalmente era de dos dormitorios pero fue refaccionada al plano actual (master suite más dormitorio chico o escritorio). impecable!,"bulnes y libertad

In [15]:
def guardarPileta(x):
    if x['pileta_en_desc'] == True:
        return x['pileta_en_desc']
    if x['pileta_en_el_titulo'] == True:
        return x['pileta_en_el_titulo']
    else:
        return False


In [16]:
#busco parrillas en el titulo y la desc
pileta_desc = df_limpio["description"].str.contains('pileta')
pileta_titl = df_limpio["title"].str.contains('pileta')

In [17]:
pileta_junto = pd.DataFrame({'pileta_en_desc':parrilla_desc,
                             'pileta_en_el_titulo':parrilla_titl
                             })
df_limpio["pileta"] = pileta_junto.apply(guardarPileta,axis=1)
df_limpio

,Unnamed: 0,Unnamed: 0.1,ID,property_type,place_name,country_name,state_name,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,price_consistent,parrilla,pileta
0,0,2,2,apartment,Mataderos,Argentina,Capital Federal,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,1309.090909,1309.090909,NaN,2.0,NaN,"2 ambientes 3er piso lateral living comedor amplio cocina separada con lavadero, baño completo y dormitorio con placard. bajas expensas, excelente ubicacion aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).",2 amb 3er piso con ascensor apto credito,True,False,False
1,1,7,7,apartment,Belgrano,Argentina,Capital Federal,"-34.5598729,-58.443362",-34.559873,-58.443362,...,3066.666667,3450.000000,NaN,1.0,NaN,"excelente monoambiente a estrenar amplio super luminoso ...piso alto cocina americana. baño completo pileta ,solarium, laundry apto profesionalproximo av del libertador muy buena zona !!!! aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).","jose hernandez 1400 monoambiente estrenar categoria , venta",True,False,False
2,2,8,8,apartment,Belgrano,Argentina,Capital Federal,"-34.5598729,-58.443362",-34.559873,-58.443362,...,3000.000000,3250.000000,NaN,2.0,NaN,"excelente dos ambientes estrenar amplio super luminoso ...piso alto cocina americana. baño completo pileta ,solarium, laundry apto profesionalproximo av del libertador muy buena zona !!!! aviso publicado por pixel inmobiliario (servicio de páginas web para inmobiliarias).","jose hernandez 1400 dos ambientes estrenar , categoria , venta",True,False,False
3,3,13,13,apartment,Palermo Soho,Argentina,Capital Federal,NaN,NaN,NaN,...,2234.000000,3723.333333,NaN,3.0,NaN,"torre i mondrian. 3 ambientes con terraza y dos baños al frente.vitraux ofrece en el ultimo nivel, piscina con solarium, un espacio con sala de reunión para uso profesional, eventos o proyecciones con un amplio foyer, sauna, gimnasio, un salón de usos múltiples con cocina y una amplia expansión. asimismo cuenta con un hall en doble altura y un amplio estar que balconea al mismo en el primer piso.así destaca la excelente relación de espacios de uso común sobre el metro propio, sobre todo en studios que rondan los 30 metros propios y pueden gozar de todos los diferentes usos especiales.",vitraux palermo,True,False,False
4,4,14,14,apartment,Palermo Soho,Argentina,Capital Federal,NaN,NaN,NaN,...,3521.428571,4770.967742,NaN,1.0,NaN,"torre ii dalí. ambiente unico divisible.vitraux ofrece en el ultimo nivel, piscina con solarium, un espacio con sala de reunión para uso profesional, eventos o proyecciones con un amplio foyer, sauna, gimnasio, un salón de usos múltiples con cocina y una amplia expansión. asimismo cuenta con un hall en doble altura y un amplio estar que balconea al mismo en el primer piso.así destaca la excelente relación de espacios de uso común sobre el metro propio, sobre todo en studios que rondan los 30 metros propios y pueden gozar de todos los diferentes usos especiales.",vitraux palermo,True,False,False
5,5,19,19,apartment,Palermo,Argentina,Capital Federal,"-34.580503566,-58.4058744847",-34.580504,-58.405874,...,3365.384615,3645.833333,NaN,3.0,NaN,excelente semipiso al contra frente en bulnes y libertador. espectacular pulmón muy luminoso con árboles. buen edificio con entrada reciclada hace dos años. hall de entrada. living comedor de muy buenas dimensiones con salida al balcón aterrazado. toilette de recpeción. master suite muy grande con vestidor y gran baño (revestimientos de marmol) también al frente. segundo dormitorio o escritorio (chico) a aire luz muy luminoso. muy linda cocina con muebles alemanes. pisos de parquet. persianas eléctricas. aislación acústica en paredes. detalles de muy buena calidad. baulera común. la unidad originalmente era de dos dormitorios pero fue refaccionada al plano actual (master suite más dormitorio chico o escritorio). impecable!,"bulnes y libertador: espectacula

In [18]:
df_limpio.groupby('place_name').describe()

ID                                                 \
                       count          mean           std      min       25%   
place_name                                                                    
Abasto                76.0    60140.631579  30005.184693  791.0    40808.25   
Agronomía             27.0    55127.333333  26907.822168  382.0    41226.50   
Almagro               849.0   54529.115430  31946.270232  98.0     28454.00   
Balvanera             549.0   59613.393443  35025.764764  48.0     30930.00   
Barracas              298.0   48670.600671  36429.703073  341.0    16287.75   
Barrio Norte          936.0   55873.690171  33556.092364  178.0    28756.50   
Belgrano              2326.0  68021.551591  32103.022017  7.0      44379.00   
Boca                  168.0   66638.875000  32143.916205  496.0    36936.50   
Boedo                 760.0   34528.656579  22925.757802  35.0     25736.75   
Caballito             1866.0  59360.515541  32073.339628  52.0     34568.25   
Capital Federal       408.0   71319.144608  31982.847668  901.0    46136.50   
Catalinas             1.0     5495.000000  NaN            5495.0   5495.00    
Centro / Microcentro  158.0   41866.753165  24715.403705  1539.0   24999.25   
Chacarita             181.0   72247.104972  30975.766885  958.0    55657.00   
Coghlan               203.0   63397.359606  38210.164279  385.0    29260.00   
Colegiales            280.0   59181.803571  33318.666238  258.0    33231.25   
Congreso              265.0   50452.215094  31664.106860  962.0    26347.00   
Constitución          169.0   49547.467456  33533.471919  251.0    17167.00   
Flores                1020.0  59228.261765  31707.991702  29.0     33234.50   
Floresta              274.0   54228.405109  29326.584486  828.0    30807.75   
Las Cañitas           126.0   37682.785714  25294.412063  38.0     16780.50   
Liniers               188.0   70562.271277  31185.819663  1407.0   49032.50   
Mataderos             125.0   67233.120000  34020.439120  2.0      38912.00   
Monserrat             434.0   55202.449309  42165.084121  411.0    8034.25    
Monte Castro          41.0    40886.048780  32919.063104  827.0    14976.00   
Nuñez                 613.0   67461.176183  33519.590489  60.0     41781.00   
Once                  137.0   52549.248175  32797.368228  4828.0   25004.00   
Palermo               2223.0  65092.782726  32247.285143  19.0     38703.00   
Palermo Chico         172.0   52422.139535  32645.369970  1276.0   22801.75   
Palermo Hollywood     430.0   71323.576744  31100.768531  266.0    49074.50   
...                      ...           ...           ...   ...          ...   
Parque Avellaneda     22.0    40074.954545  28450.931476  4287.0   23229.50   
Parque Centenario     58.0    43670.172414  29189.589747  390.0    20592.25   
Parque Chacabuco      78.0    53926.333333  34942.615044  5195.0   21133.25   
Parque Chas           34.0    30547.235294  27174.818836  340.0    17257.25   
Parque Patricios      214.0   73185.196262  29960.813455  4415.0   54685.25   
Paternal              94.0    47803.787234  35559.159023  351.0    16389.50   
Pompeya               16.0    50085.375000  30742.646151  5717.0   32114.25   
Puerto Madero         551.0   56056.312160  33772.893176  244.0    24481.50   
Recoleta              1313.0  56629.476009  34454.671784  200.0    28475.00   
Retiro                200.0   54264.845000  31612.025774  838.0    28457.75   
Saavedra              411.0   63930.063260  27508.845437  270.0    46111.50   
San Cristobal         486.0   41758.839506  26518.668085  696.0    25912.75   
San Nicolás           146.0   54721.993151  34933.954361  876.0    27790.25   
San Telmo             1064.0  71466.953008  35319.639188  403.0    48030.75   
Tribunales            64.0    55297.984375  32482.619052  4683.0   31405.75   
Velez Sarsfield       10.0    63606.100000  35094.320986  7259.0   48839.75   
Versalles             18.0    45574.055556  40024.797184  819.0    13478.25 